In [1]:
# Importing Dependencies
import pandas as pd
import requests
import time as tm
from datetime import datetime, timedelta

#Import API Key
from api_key import wx_api_key

### Creating List of Dates

In [2]:
# Creating List of Dates
dates = []

# Assuming 365 Days in a Year
number_of_days = 365

# Inputting Start Date
start_date_obj = datetime(2015, 1, 1)

# For Loop to Create List of Dates
for day in range(number_of_days):
    new_date = start_date_obj + timedelta(day)
    new_date = new_date.strftime("%Y-%m-%d")
    dates.append(new_date)

### Creating DataFrame of New York Airports

In [3]:
#read the csv files in
airport = "../Resources/airports.csv"

#create df for airports
airport_df = pd.read_csv(airport)

# Locating Airports in the US that have Small Airports
sm_arprts = airport_df.loc[(airport_df['iso_country']=='US') & (airport_df['type']=='small_airport') & (airport_df['iso_region']=='US-NY')]
                                                                
# Dropping NaN in IATA                                                              
sm_arprts = sm_arprts.dropna(subset=['iata_code'])

# Locating Airports in the US that have Medium Airports
md_arprts = airport_df.loc[(airport_df['iso_country']=='US') & (airport_df['type']=='medium_airport') & (airport_df['iso_region']=='US-NY')]
                                                                
# Dropping NaN in IATA                                                              
md_arprts = md_arprts.dropna(subset=['iata_code'])

# Locating Airports in the US that have Medium Airports
lg_arprts = airport_df.loc[(airport_df['iso_country']=='US') & (airport_df['type']=='large_airport') & (airport_df['iso_region']=='US-NY')]
                                                                
# Dropping NaN in IATA                                                              
lg_arprts = lg_arprts.dropna(subset=['iata_code'])

# Concatinating DataFrames into One
arprts = pd.concat([sm_arprts, md_arprts, lg_arprts])

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
29449,3431,KBUF,large_airport,Buffalo Niagara International Airport,42.940498,-78.732201,728.0,NaN,US,US-NY,Buffalo,yes,KBUF,BUF,BUF,NaN,https://en.wikipedia.org/wiki/Buffalo_Niagara_...,NaN
30858,3622,KJFK,large_airport,John F Kennedy International Airport,40.639801,-73.778900,13.0,NaN,US,US-NY,New York,yes,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild"
30997,3643,KLGA,large_airport,La Guardia Airport,40.777199,-73.872597,21.0,NaN,US,US-NY,New York,yes,KLGA,LGA,LGA,https://www.laguardiaairport.com/,https://en.wikipedia.org/wiki/LaGuardia_Airport,"Manhattan, New York City, NYC, Glenn H. Curtis..."
32986,3855,KROC,large_airport,Greater Rochester International Airport,43.118900,-77.672401,559.0,NaN,US,US-NY,Rochester,yes,KROC,ROC,ROC,NaN,https://en.wikipedia.org/wiki/Greater_Rocheste...,NaN
33236,3913,KSYR,large_airport,Syracuse Hancock International Airport,43.111198,-76.106300,421.0,NaN,US,US-NY,Syracuse,yes,KSYR,SYR,SYR,http://www.syrairport.org/,https://en.wikipedia.org/wiki/Syracuse_Hancock...,NaN


### Creating List of IATA Codes

In [8]:
iata_codes = arprts['iata_code']

# List of Major Airports in New York
locs = ['JFK','LGA']

# # Looping Through Iata Codes
# for code in iata_codes:
#     locs.append(code)

###  Perform API Calls

In [9]:
# Base URL
base_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"

# WorldWeather Online Has Two Formats with XML as the Default
format_output = 'json'

# Time Interval, Set to 6 Hourly Intervals
tp = 6

# Include location
incLoc = 'yes'

# Building Query URL
query_url = f"{base_url}key={wx_api_key}&format={format_output}&tp={tp}&includelocation={incLoc}"

In [10]:
# Creating Empty Lists
date_list = []
time_list = []
loc_list = []
lat =[]
lng = []
tempF = []
DewPointF = []
windspeedMiles = []
WindGustMiles = []
winddir16Point = []
weatherDesc = []
precipInches = []
humidity = []
visibilityMiles = []
pressure = []
cloudcover = []
HeatIndexF = []
WindChillF = []
totalSnow_cm = []

# Using a For Loop to Make Successive API Calls
record_count = 1

# Using a For Loop to Make Successive API Calls
print('Beginning Data Retrieval')
print('------------------------')

# Loop Through Each Time Interval (*Hard Coded for 6 Hourly Data*)
for time in range(4):
    # Loop Through Dates        
    for date in dates:
        # Loop Through IATA Codes
        for loc in locs:
            # Making Requests
            if record_count >= 50:
                tm.sleep(60)
                record_count = 0

            response = requests.get(f'{query_url}&date={date}&q={loc}').json()
            record_count += 1
            
            # Using Try/Except to Grab Variables if Available
            try:
                print(f'Processing Time Interval {time} for {date} for {loc}')
                loc_list.append(loc)
                date_list.append(date)
                time_list.append(response['data']['weather'][0]['hourly'][time]['time'])
                lat.append(response['data']['nearest_area'][0]['latitude'])
                lng.append(response['data']['nearest_area'][0]['longitude'])
                totalSnow_cm.append(response['data']['weather'][0]['totalSnow_cm'])
                tempF.append(response['data']['weather'][0]['hourly'][time]['tempF'])
                DewPointF.append(response['data']['weather'][0]['hourly'][time]['DewPointF'])
                windspeedMiles.append(response['data']['weather'][0]['hourly'][time]['windspeedMiles'])
                WindGustMiles.append(response['data']['weather'][0]['hourly'][time]['WindGustMiles'])
                winddir16Point.append(response['data']['weather'][0]['hourly'][time]['winddir16Point'])
                weatherDesc.append(response['data']['weather'][0]['hourly'][time]['weatherDesc'][0]['value'])
                precipInches.append(response['data']['weather'][0]['hourly'][time]['precipInches'])
                humidity.append(response['data']['weather'][0]['hourly'][time]['humidity'])
                visibilityMiles.append(response['data']['weather'][0]['hourly'][time]['visibilityMiles'])
                pressure.append(response['data']['weather'][0]['hourly'][time]['pressure'])
                cloudcover.append(response['data']['weather'][0]['hourly'][time]['cloudcover'])
                HeatIndexF.append(response['data']['weather'][0]['hourly'][time]['HeatIndexF'])
                WindChillF.append(response['data']['weather'][0]['hourly'][time]['WindChillF'])
            except KeyError:
                print('Day not found. Skipping...')
print('------------------------')
print('Data Retrieval Complete')
print('------------------------')

Beginning Data Retrieval
------------------------
Processing Time Interval 0 for 2015-01-01 for JFK
Processing Time Interval 0 for 2015-01-01 for LGA
Processing Time Interval 0 for 2015-01-02 for JFK
Processing Time Interval 0 for 2015-01-02 for LGA
Processing Time Interval 0 for 2015-01-03 for JFK
Processing Time Interval 0 for 2015-01-03 for LGA
Processing Time Interval 0 for 2015-01-04 for JFK
Processing Time Interval 0 for 2015-01-04 for LGA
Processing Time Interval 0 for 2015-01-05 for JFK
Processing Time Interval 0 for 2015-01-05 for LGA
Processing Time Interval 0 for 2015-01-06 for JFK
Processing Time Interval 0 for 2015-01-06 for LGA
Processing Time Interval 0 for 2015-01-07 for JFK
Processing Time Interval 0 for 2015-01-07 for LGA
Processing Time Interval 0 for 2015-01-08 for JFK
Processing Time Interval 0 for 2015-01-08 for LGA
Processing Time Interval 0 for 2015-01-09 for JFK
Processing Time Interval 0 for 2015-01-09 for LGA
Processing Time Interval 0 for 2015-01-10 for JFK


Processing Time Interval 0 for 2015-03-23 for LGA
Processing Time Interval 0 for 2015-03-24 for JFK
Processing Time Interval 0 for 2015-03-24 for LGA
Processing Time Interval 0 for 2015-03-25 for JFK
Processing Time Interval 0 for 2015-03-25 for LGA
Processing Time Interval 0 for 2015-03-26 for JFK
Processing Time Interval 0 for 2015-03-26 for LGA
Processing Time Interval 0 for 2015-03-27 for JFK
Processing Time Interval 0 for 2015-03-27 for LGA
Processing Time Interval 0 for 2015-03-28 for JFK
Processing Time Interval 0 for 2015-03-28 for LGA
Processing Time Interval 0 for 2015-03-29 for JFK
Processing Time Interval 0 for 2015-03-29 for LGA
Processing Time Interval 0 for 2015-03-30 for JFK
Processing Time Interval 0 for 2015-03-30 for LGA
Processing Time Interval 0 for 2015-03-31 for JFK
Processing Time Interval 0 for 2015-03-31 for LGA
Processing Time Interval 0 for 2015-04-01 for JFK
Processing Time Interval 0 for 2015-04-01 for LGA
Processing Time Interval 0 for 2015-04-02 for JFK


Processing Time Interval 0 for 2015-06-13 for LGA
Processing Time Interval 0 for 2015-06-14 for JFK
Processing Time Interval 0 for 2015-06-14 for LGA
Processing Time Interval 0 for 2015-06-15 for JFK
Processing Time Interval 0 for 2015-06-15 for LGA
Processing Time Interval 0 for 2015-06-16 for JFK
Processing Time Interval 0 for 2015-06-16 for LGA
Processing Time Interval 0 for 2015-06-17 for JFK
Processing Time Interval 0 for 2015-06-17 for LGA
Processing Time Interval 0 for 2015-06-18 for JFK
Processing Time Interval 0 for 2015-06-18 for LGA
Processing Time Interval 0 for 2015-06-19 for JFK
Processing Time Interval 0 for 2015-06-19 for LGA
Processing Time Interval 0 for 2015-06-20 for JFK
Processing Time Interval 0 for 2015-06-20 for LGA
Processing Time Interval 0 for 2015-06-21 for JFK
Processing Time Interval 0 for 2015-06-21 for LGA
Processing Time Interval 0 for 2015-06-22 for JFK
Processing Time Interval 0 for 2015-06-22 for LGA
Processing Time Interval 0 for 2015-06-23 for JFK


ConnectionError: HTTPSConnectionPool(host='api.worldweatheronline.com', port=443): Max retries exceeded with url: /premium/v1/past-weather.ashx?key=c2f168cf23884601ae922017212901&format=json&tp=6&includelocation=yes&date=2015-09-03&q=LGA (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001939E8FB6A0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

### Creating and Cleaning DataFrame

In [11]:
 # Creating DataFrame
historical_wx_df = pd.DataFrame({
    'Airport Code': loc_list,
    'Date': date_list,
    'Time': time_list,
    'Latitude': lat,
    'Longitude': lng,
    'Weather Description': weatherDesc,
    'Temperature (F)': tempF,
    'Heat Index (F)': HeatIndexF,
    'Wind Chill (F)': WindChillF,
    'Dew Point (F)': DewPointF,
    'Wind Speed (mph)': windspeedMiles,
    'Wind Gusts (mph)': WindGustMiles,
    'Wind Direction': winddir16Point,
    'Precipitation (in)': precipInches,
    'Humidity (%)': humidity,
    'Visibility (mi)': visibilityMiles,
    'Pressure (mb)': pressure,
    'Cloud Coverage (%)': cloudcover,
    'Snow Total (cm)': totalSnow_cm
})
historical_wx_df

,Airport Code,Date,Time,Latitude,Longitude,Weather Description,Temperature (F),Heat Index (F),Wind Chill (F),Dew Point (F),Wind Speed (mph),Wind Gusts (mph),Wind Direction,Precipitation (in),Humidity (%),Visibility (mi),Pressure (mb),Cloud Coverage (%),Snow Total (cm)
0,JFK,2015-01-01,0,40.640,-73.779,Clear,30,31,20,17,14,19,WSW,0.0,57,6,1022,19,0.0
1,LGA,2015-01-01,0,40.776,-73.873,Clear,28,28,19,15,11,16,WSW,0.0,58,6,1021,13,0.0
2,JFK,2015-01-02,0,40.640,-73.779,Clear,37,37,28,25,15,24,WSW,0.0,62,6,1017,6,0.0
3,LGA,2015-01-02,0,40.776,-73.873,Clear,35,35,26,23,12,21,WSW,0.0,63,6,1017,6,0.0
4,JFK,2015-01-03,0,40.640,-73.779,Overcast,35,36,29,26,9,14,NNE,0.0,69,6,1032,100,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,JFK,2015-09-01,0,40.640,-73.780,Partly cloudy,70,76,72,65,9,14,WSW,0.0,79,6,1018,32,0.0
487,LGA,2015-09-01,0,40.780,-73.870,Partly cloudy,69,73,70,64,7,12,WSW,0.0,81,5,1018,33,0.0
488,JFK,2015-09-02,0,40.640,-73.780,Clear,73,97,84,79,3,3,WSW,0.0,85,6,1017,0,0.0
489,LGA,2015-09-02,0,40.776,-73.873,Clear,71,86,80,75,3,4,WSW,0.0,87,6,1017,0,0.0


### Creating CSV File

In [12]:
# Creating CSV File
historical_wx_df.to_csv('../Resources/historical_wx.csv')

### Reading CSV File

In [13]:
 # Reading in CSV File
filepath = '../Resources/historical_wx.csv'
f = pd.read_csv(filepath)
df = pd.DataFrame(f)
df = df.drop(columns=['Unnamed: 0']) # Dropping Column

df.head()

,Airport Code,Date,Time,Latitude,Longitude,Weather Description,Temperature (F),Heat Index (F),Wind Chill (F),Dew Point (F),Wind Speed (mph),Wind Gusts (mph),Wind Direction,Precipitation (in),Humidity (%),Visibility (mi),Pressure (mb),Cloud Coverage (%),Snow Total (cm)
0,JFK,2015-01-01,0,40.640,-73.779,Clear,30,31,20,17,14,19,WSW,0.0,57,6,1022,19,0.0
1,LGA,2015-01-01,0,40.776,-73.873,Clear,28,28,19,15,11,16,WSW,0.0,58,6,1021,13,0.0
2,JFK,2015-01-02,0,40.640,-73.779,Clear,37,37,28,25,15,24,WSW,0.0,62,6,1017,6,0.0
3,LGA,2015-01-02,0,40.776,-73.873,Clear,35,35,26,23,12,21,WSW,0.0,63,6,1017,6,0.0
4,JFK,2015-01-03,0,40.640,-73.779,Overcast,35,36,29,26,9,14,NNE,0.0,69,6,1032,100,0.0
